# Custom ChatGPT with LangChain

The official ChatGPT web app is great, but it might not be tailored to our specific needs. A custom ChatGPT app provides several advantages over using the prebuilt ChatGPT app.
## Advantages of Using a Custom ChatGPT App
**1)Increased Security:-** You can implement security measures, designed to meet your company's needs This means, you can control who has access to the app and how the app is used. This can help to protect your company's data. for example, you could implement two-factor authentication, restrict access to the app based on the user's role, or encrypt the stored data. You can also use a firewall to block unauthorized access to the app.

**2)Improved User Experience:-** You can personalize the app according to your user needs. This means you can customize the app's layout, branding, and functionality to match your company's style and requirements

**3)Greater Flexibility** to add new features or functionality as needed. Because we are not limited to the features available in the pre-built ChatGPT app. for example, you can use another model like Llama2(Meta) which is open source, or another one from the Hugging Face Hub. You can add new features to the app as your needs change, or you can modify the app's functionality to meet the specific needs of your users. for example, you can add a feature that allows users to track their progress on a project, or you can connect a generative text model.

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = True)

In this app, we will use a chat model, Which means,the exchange of messages back and forth. Instead of PromptTemplate, We will use **ChatPromptTemplate**.

We will add memory to our custom ChatGPT app. Memory is the ability to store information about past interactions with the LLM. We can create an instance of memory and incorporate it into a chain. The general goal of memory is to store some data inside the chain, so we can use it in the future when we run the same chain again. LangChain provides us with many types of memory. Each has its own parameters, its own return types, and is useful in different scenarios.

For Chat models, we will use **ConversationBufferMemory** class. Its goal is to keep a list of all the chat messages in a buffer and pass those into the prompt template.

**MessagesPlaceholder** is a prompt template that allows you to insert a list of history messages during formatting, and it is useful when you are using ConversationBufferMemory to store the chat history.

if we take a look at the original ChatGPT app or Bard, we can notice that all past conversations will be saved there. If we interrupt the session and return later, we can continue the same conversation. We use a special langchain component named **FileChatMessageHistory**to implement this feature in our LLM app. We save the chat messages to a file before exiting, and when starting the app again, it will load the previous messages from that file. Note:- Instead of a file, we can save the chat history to more robust external databases too, like Postgres, MongoDB, and so on.

In [ ]:
#1. imports

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 1)


history=FileChatMessageHistory('chat_history456.json')  #All the chat messages history will be saved to this file


#2 memory object
memory = ConversationBufferMemory(
    memory_key = 'chat_history', # It means that the chat memory will be stored in the chat_history key. This key will be used in ChatPromptTemplate.
    chat_memory=history,
    return_messages = True   # It tells the memory to return the chat history as a list of messages instead of a string.
)

prompt = ChatPromptTemplate(
    input_variable = ['content'],
    messages=[
    SystemMessage(content='You are a chatbot having a conversation with a human.'), 
    MessagesPlaceholder(variable_name='chat_history'), #where the memory will be stored. It has to be added here..after the SystemMessage and before the HumanMessagePromptTemplate.
    HumanMessagePromptTemplate.from_template('{content}')
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,  # adding memory to the chain
    verbose =True
)



while True:
    content = input('Your prompt: ')
    if content in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break

    response = chain.invoke({'content': content})
    print(response['text'])
    print('-'*50)

Your prompt:  where is it located?




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: What is the capital of india
AI: The capital of India is New Delhi.
Human: same famous tourist places over there?
AI: Yes, there are many famous tourist places in New Delhi, such as the Red Fort, India Gate, Qutub Minar, Lotus Temple, Akshardham Temple, Humayun's Tomb, and Rashtrapati Bhavan (the President's Residence). Delhi also has vibrant markets like Chandni Chowk and Connaught Place where you can shop and experience local culture.
Human: where is it located?

> Finished chain.
New Delhi is located in the northern part of India. It is part of the National Capital Territory of Delhi, which is surrounded by the state of Haryana on three sides and Uttar Pradesh on the east.
--------------------------------------------------
